<a href="https://colab.research.google.com/github/gabrielcassimiro17/build-with-ai/blob/main/Build_With_AI_Gemini_GDG_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Gemini no Colab

## Passo 1 criar uma conta na Google AI Studio


<a class="button button-primary" href="https://aistudio.google.com/" target="_blank" rel="noopener noreferrer">Google AI Studio</a>



## Passo 2 configurar API Key


Antes de poder usar a API da Gemini, você deve obter uma chave de API. Se você ainda não tem uma, crie uma chave com um clique no Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Obeter uma chave de API</a>



## Passo 3: Setup API Key no notebook

No Colab, adicione a chave ao gerenciador de segredos sob o ícone "🔑" no painel esquerdo. Dê a ela o nome GOOGLE_API_KEY.

# Gemini Basics

## Setup

### Instalando o SDK Python


O SDK do Python para a API Gemini está contido no pacote [`google-generativeai`](https://pypi.org/project/google-generativeai/). Instale a dependência usando pip:

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 12.4 MB/s eta 0:00:00


### Importando libs

Importando as libs necessárias

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Used to securely store your API key
from google.colab import userdata

### Configure sua chave de API no Notebook


Dando acesso ao Colab para sua chave de API e acessando ela via código

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Listar modelos

Agora você está pronto para chamar a API Gemini. Use `list_models` para ver os modelos Gemini disponíveis:

* `gemini-pro`: otimizado para prompts somente de texto.
* `gemini-pro-vision`: otimizado para prompts de texto e imagens.

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Nota: Para informações detalhadas sobre os modelos disponíveis, incluindo suas capacidades e limites de taxa, consulte [Modelos Gemini](https://ai.google.dev/models/gemini). Existem opções para solicitar [aumentos no limite de taxa](https://ai.google.dev/docs/increase_quota). O limite de taxa para os modelos Gemini-Pro é de 60 solicitações por minuto (RPM).

O pacote `genai` também suporta a família de modelos PaLM, mas somente os modelos Gemini suportam as capacidades genéricas e multimodais do método `generateContent`.

## Gerar texto a partir de entradas de texto

Para prompts somente com texto, use o modelo `gemini-pro`:


In [ ]:
### Declarando o modelo



O método `generate_content` pode lidar com uma ampla variedade de casos de uso, incluindo conversas de múltiplas interações e entrada multimodal, dependendo do que o modelo subjacente suporta. Os modelos disponíveis suportam apenas texto e imagens como entrada, e texto como saída.

No caso mais simples, você pode passar uma string de prompt para o método <a href="https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content"><code>GenerativeModel.generate_content</code></a>:

In [ ]:
%%time


#### usando o modelo

Em casos simples, o acessor `response.text` é suficiente. Para exibir texto Markdown formatado, use a função `to_markdown`:


In [ ]:
to_markdown(response.text)

Se a API não retornou um resultado, use `GenerateContentResponse.prompt_feedback` para verificar se o prompt foi bloqueado devido a questões de segurança.


In [ ]:
#### Prompt feedback

O Gemini pode gerar várias respostas possíveis para um único prompt

Essas respostas possíveis são chamadas de `candidates` (candidatos) e você pode analisá-las para selecionar a mais adequada como resposta final.

Visualize os candidatos de resposta com `GenerateContentResponse.candidates`: Documentação da API: [https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse#candidates](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse#candidates)


In [ ]:
response.candidates

Por padrão, o modelo retorna uma resposta após concluir todo o processo de geração. Você também pode transmitir a resposta à medida que ela é gerada, e o modelo retornará partes da resposta assim que forem geradas.

Para transmitir respostas, use `GenerativeModel.generate_content(..., stream=True)` Documentação da API: [https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content).


In [ ]:
%%time

### stream



In [ ]:
for chunk in response:
  print(chunk.text)
  print("_"*80)

## Gere texto a partir de entradas de imagem e texto

Gemini oferece um modelo multimodal (`gemini-pro-vision`) que aceita tanto texto quanto imagens como entradas. A API `GenerativeModel.generate_content` é projetada para lidar com prompts multimodais e retorna uma saída de texto.

Vamos incluir uma imagem:

In [ ]:
!curl -o image.jpg https://t0.gstatic.com/licensed-image?q=tbn:ANd9GcQ_Kevbk21QBRy-PgB4kQpS79brbmmEG7m3VOTShAn4PecDU5H5UxrJxE3Dw1JiaG17V88QIol19-3TM2wCHw

In [ ]:
import PIL.Image

img = PIL.Image.open('image.jpg')
img

Use o modelo `gemini-pro-vision` e passe a imagem para o modelo com `generate_content`.

In [ ]:
#### declarando o modelo vision


In [ ]:
### generate content


In [ ]:
to_markdown(response.text)

Para fornecer tanto texto quanto imagens em um prompt, passe uma lista contendo as strings e imagens:

In [ ]:
#### Prompt e imagem

prompt = """Escreva a receita para o prato presente na imagem, em portugês."""

response = model.generate_content([prompt, img], stream=True)
response.resolve()

In [ ]:
to_markdown(response.text)

## Contar tokens

Modelos de linguagem grandes possuem uma janela de contexto, e o comprimento do contexto é frequentemente medido em termos do **número de tokens**. Com a API Gemini, você pode determinar o número de tokens para qualquer objeto `glm.Content`. No caso mais simples, você pode passar uma string de consulta para o método `GenerativeModel.count_tokens` da seguinte forma:

In [ ]:
#### count tokens

Da mesma forma, você pode verificar `token_count` para a resposta do modelo:

In [ ]:
#### count tokens

# LangChain

### O que é LangChain

LangChain é um framework para o desenvolvimento de aplicações potencializadas por LLMs.

LangChain simplifica cada estágio do ciclo de vida da aplicação LLM:

- Desenvolvimento: Construa suas aplicações usando os blocos de construção e componentes de código aberto do LangChain. Comece rapidamente usando integrações de terceiros e Templates.
- Produção: Use o LangSmith para inspecionar, monitorar e avaliar suas cadeias, de modo que você possa otimizar continuamente e implantar com confiança.
- Implantação: Transforme qualquer cadeia em uma API com o LangServe.

### Vantagens

- LangChain permite conectar LLMs a fontes de dados externas para aplicativos de NLP personalizados
- Fornece módulos para integrar LLMs, fontes de dados e armazenamento
- Possibilita a construção de aplicativos de IA conversacional, sumarização, busca e outras capacidades de NLP
- Ajuda os usuários a aproveitar a NLP e os LLMs para várias aplicações específicas da indústria de maneira eficiente e eficaz

In [ ]:
%pip install --quiet  langchain-google-genai langchain pip install faiss-cpu

Setup de variável de ambiente para uso do LangChain:

In [ ]:
from google.colab import userdata
import os


os.environ["GOOGLE_API_KEY"] =userdata.get('GOOGLE_API_KEY')

## Chamada Simples

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

####

Podemos usar streaming aqui também, com cada bloco de tokens sendo retornado assim que for gerado.

Dessa forma conseguimos dar uma resposta mais rápida ao usuário que vai acompanhando a geração dos tokens em tempo real

In [ ]:
#### Streaming com langchain



## Construindo um caso de resumo de textos

Para o caso de resumo de um longo texto vamos usar o modelo ***Gemini 1.5 Pro***.

Esse modelo é capaz de processar até **1 Milhão de tokens** em um único prompt.


Para isso vamos usar o livro **Dom Casmurro** como exemplo.

### Baixando o livro via link direto

In [ ]:
import requests

# URL to a public domain text by Machado de Assis, for example, "Dom Casmurro"
url = "https://www.gutenberg.org/cache/epub/55752/pg55752.txt"

response = requests.get(url)
livro = response.text

# Display the first 1000 characters to verify
print(livro[1500:3000])


Mostrando Tamanho da string contendo o livro

In [ ]:
print(len(livro))

Podemos também usar a função de contar tokens ***(count_tokens)*** para ver tamanho do livro que queremos processar.

In [ ]:
#### Gemini 1.5


#### count tokens

### Passando livro no prompt para resumo

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

### Carregando gemini-1.5-pro-latest

### definindo o prompt

#### chamando LLM

#### mostrando resultado


## Usando dados no prompt

Podemos também fazer **perguntas sobre o texto de contexto** que estamos passando para LLM. Dessa forma a **resposta será baseada nesse contexto** passado.

Essa aplicação permite **mostrar para a LLM dados não vistos no seu treinamento**, fornecendo assim contexto para que ela responda perguntas desses temas.


In [ ]:

pergunta= """
Em 'Dom Casmurro', Bentinho descreve um hábito peculiar de Capitu que ele observa repetidamente.
Qual é esse hábito e como ele interpreta essa característica dela?
"""

### Formatar prompt

In [ ]:
### Invoke LLM


### mostrar resultado

Porém nem todas as LLMs suportam uma quantidade tão grande de tokens e quanto mais tokens passados em um prompt mais caro será a chamada.

Quando falamos de contexto de LLMs sempre temos que considerar o tradeoff:

- Qualidade/Capacidade
- Custo
- Tempo de resposta  



LLMs mais "potentes" tendem a ter um custo mais por token e demorar mais a responder. Por isso é necessário utilizar LLMs "menos potentes" sempre que possível.



---



Testando com o Gemini Pro 1.0:

In [ ]:
pergunta= """
Em 'Dom Casmurro', Bentinho descreve um hábito peculiar de Capitu que ele observa repetidamente.
Qual é esse hábito e como ele interpreta essa característica dela?
"""

### Carregando gemini-pro

### definindo o prompt

#### chamando LLM

#### mostrando resultado

In [ ]:
### chamando direto da lib google

Por esse motivo podemos usar a técnica de RAG:

### Embeddings

O conceito de embeddings é fundamental para o funcionamento da técnica de RAG (Retrieval-Augmented Generation), uma abordagem híbrida que combina a recuperação de informações e a geração de texto.

- **Uso de Embeddings em RAG:** Os embeddings são aplicados para representar e indexar grandes quantidades de dados textuais.
- **Facilitação da Busca:** Eles permitem que o modelo de RAG busque de forma eficiente os trechos mais relevantes de acordo com o contexto ou pergunta específica.



---



Usando Embeddings da Google

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

### inicializando embeddings

### aplicando embeddings

print(vector[:5])
print(f"Tamanho do vetor: {len(vector)}")

Podemos demonstrar como RAG funciona com esse exemplo simples comparando 4 frases:

In [ ]:
import numpy as np

vector_1 = embeddings.embed_query("Eu amo Inteligência artificial")
vector_2 = embeddings.embed_query("Flores tropicais florescem predominantemente no verão.")
vector_3 = embeddings.embed_query("Alan Turing desenvolveu o conceito de computação moderna")
vector_4 = embeddings.embed_query("Eu gosto muito de Inteligência artificial")

def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

similarity_1_2 = cosine_similarity(vector_1, vector_2)

similarity_1_3 = cosine_similarity(vector_1, vector_3)

similarity_1_4 = cosine_similarity(vector_1, vector_4)

# Output the results
print(f"Cosine Similarity between vector 1 and vector 2: {similarity_1_2}")
print(f"Cosine Similarity between vector 1 and vector 3: {similarity_1_3}")
print(f"Cosine Similarity between vector 1 and vector 3: {similarity_1_4}")


### Vector DBs

Bancos de dados vetoriais são cruciais para o RAG porque armazenam e gerenciam embeddings de forma eficiente. Esse processo de recuperação é essencial, pois enriquece a entrada para o modelo gerador, aumentando a precisão e a riqueza das respostas que ele pode produzir.

- **Armazenamento em Bancos de Dados Vetoriais:** Os embeddings são armazenados em bancos de dados vetoriais, otimizando a gestão de grandes volumes de dados.
- **Recuperação Rápida de Informações:** Esses bancos permitem que o modelo RAG realize buscas rápidas baseadas em similaridade vetorial.


**O chunking de documentos** é o processo de dividir textos grandes em segmentos menores, chamados chunks, para facilitar o gerenciamento, aumentar a qualidade de recuperação desses textos e permitir a redução da quantidade de tokens passados no prompt.

Esses segmentos são então inseridos em um banco de dados vetorial, permitindo que cada pedaço seja indexado e recuperado eficientemente com base em sua semelhança semântica.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

### Definindo text_splitter

### aplicando text splitter

print(len(docs))
print(len(docs[14].page_content))
print(docs[10])
print(docs[11])
print(docs[12])

Aqui vamos usar o Vector DB FAISS, que armazena os embeddings em memória.

Para experimentação rápida ele é uma boa alternativa, mas para aplicações em escala que você precisa persistir os dados, é recomendado usar alguma versão em cloud ou fazer o deploy do seu próprio.

In [ ]:
from langchain.vectorstores import FAISS

### Vetorizando para o FAISS
vector_store = FAISS.from_documents(docs, embeddings)

retriever = vector_store.as_retriever()

retriever.search_kwargs["fetch_k"] = 10
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 3


question = "Qual era a cor do vestido que Capitu usava?"
context = retriever.get_relevant_documents(question)
context

### RAG

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")


In [ ]:
#### Pergunta
pergunta= "Qual era a cor do vestido que Capitu usava?"


### Obter contexto


### Formatar prompt
prompt = f"""



"""


model.count_tokens(prompt)



In [ ]:
response = llm.invoke(prompt)
response.content

In [ ]:
contexto = retriever.get_relevant_documents(question)
contexto

## Imagens com Langchain

Podemos também chamar os modelos multi-modais usando LangChain.


Aqui podemos passar a imagem ou diretamente a url da imagem durante a chamada do modelo.

In [ ]:
from langchain_core.messages import HumanMessage

llm_vision = ChatGoogleGenerativeAI(model="gemini-pro-vision")

image_url = "https://t0.gstatic.com/licensed-image?q=tbn:ANd9GcQ_Kevbk21QBRy-PgB4kQpS79brbmmEG7m3VOTShAn4PecDU5H5UxrJxE3Dw1JiaG17V88QIol19-3TM2wCHw"

prompt =  "Create an in depth description of the image"

product_msg = HumanMessage(
    content=[
        {
            "type": "text",
            "text": prompt,
        },
        {"type": "image_url", "image_url": image_url},
    ]
)



In [ ]:
prod_output = llm_vision.invoke([product_msg])

print(prod_output.content)

## Prompt Chains

As "chains" do LangChain são cadeias de componentes que combinam várias habilidades e serviços de LLMs para resolver problemas complexos de processamento de linguagem natural.

A grande vantagem das chains do LangChain é a **modularidade**, permitindo que desenvolvedores construam aplicações customizadas conectando diferentes módulos, como recuperação de informações, entendimento de texto e geração de linguagem.

Isso resulta em soluções mais robustas, flexíveis e escaláveis, adaptáveis a uma ampla gama de casos de uso específicos da indústria.

https://python.langchain.com/docs/expression_language/why/

### Exemplo Básico

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-pro")

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

#### Create Chain



### Invoke Chain



### RAG Chain

Setup de componentes:

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

llm = ChatGoogleGenerativeAI(model="gemini-pro")

template = """


"""

prompt = ChatPromptTemplate.from_template(template)

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)


Construção da Chain

In [ ]:
#### Chain


chain.invoke("Qual a cor do olho de capitu?")

Podemos também chamar um componente diretamente

In [ ]:
### Invoke retriever

A função Debug do LangChain permite que tenhamos visibilidade do que está acontecendo dentro das etapas da chain.

In [ ]:
import langchain

langchain.deubg = True

In [ ]:
### Invoke

In [ ]:
langchain.deubg = False

# Casos de Uso

## Rag na Wikipedia

Criar um chatbot capaz de responder perguntas sobre o tema presente nessa página da Wikipedia:

Deve responder perguntas como:

" Quais modelos do Gemini estão disponíveis?"  
" Quais modelos da Anthropic existem?"  
" Quem é o CEO do twitter?"    
" O Twitter tem alguma llm?"    

In [ ]:
%pip install --upgrade --quiet  wikipedia

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wikipedia.run("Sam Altman")

In [ ]:
ai_personalities_companies = [
    "Sam Altman",
    "Demis Hassabis",
    "Geoffrey Hinton",
    "Andrew Ng",
    "Elon Musk",
    "DeepMind",
    "Meta AI",
    "OpenAI",
    "Google Brain",
    "Google Gemini",
    "Claude 3",
    "Llama 3",
    "GPT 4",
    "Grok"
]

# Initialize a list to store summaries
summaries = []

# Loop through the list and search for each entry
for entry in ai_personalities_companies:
    summary = wikipedia.run(entry)
    summaries.append(summary)  # Append the result to the summaries list



In [ ]:
summaries[0]

In [ ]:
##### Desenvolva o código aqui:





## Criação de posts com base em fotos

Crie um post para uma rede social promovendo o prato que estão nas fotos

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt


# Function to download and display images
def download_display_images(urls):
    plt.figure(figsize=(15, 10))  # Set the figure size
    for i, url in enumerate(urls):
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            response = requests.get(url, headers=headers)
            img = Image.open(BytesIO(response.content))
            plt.subplot(1, len(urls), i + 1)  # Subplot for each image
            plt.imshow(img)
            plt.axis('off')  # Turn off axis numbers and ticks
        except Exception as e:
            print(f"Failed to load image from {url}: {e}")
    plt.show()



def display_text_image(text, image_url):
    """
    Displays text and an image side by side using matplotlib.

    Parameters:
    - text (str): The text to display.
    - image_url (str): The URL of the image to display.
    """
    # Load the image from the URL
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))

    # Set up the figure and axes
    fig, axs = plt.subplots(1, 2, figsize=(15, 7))

    # Display the image on the left
    axs[0].imshow(img)
    axs[0].axis('off')  # Turn off axis for image

    # Display the text on the right
    axs[1].text(0.5, 0.5, text, fontsize=12, ha='center', va='center', wrap=True)
    axs[1].axis('off')  # Turn off axis for text

    # Show the plot
    plt.show()


In [ ]:
# URLs of images
image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/a/a3/Eq_it-na_pizza-margherita_sep2005_sml.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e9/Egyptian_food_Koshary.jpg/640px-Egyptian_food_Koshary.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/8/89/Tomato_je.jpg/640px-Tomato_je.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/f/f9/My_food_platter.jpg/640px-My_food_platter.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/7/71/Pizza_with_food_waste.jpg/640px-Pizza_with_food_waste.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Yakisoba_%28Japanese_style_chow_mein%29_%E7%84%BC%E3%81%8D%E3%81%9D%E3%81%B0.jpg/640px-Yakisoba_%28Japanese_style_chow_mein%29_%E7%84%BC%E3%81%8D%E3%81%9D%E3%81%B0.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Fast_Food_in_Lusaka_10.jpg/640px-Fast_Food_in_Lusaka_10.jpg",
]

In [ ]:
# Call the function with the list of URLs
download_display_images(image_urls)

In [ ]:
# Example usage:
text = "This is an example of displaying text and an image side by side."

display_text_image(text, image_urls[0])

In [ ]:
##### Desenvolva o código aqui:





# Recursos e próximos passos

Documentações:
- https://ai.google.dev/gemini-api/docs/get-started/python
- https://python.langchain.com/docs/expression_language/get_started/
- https://python.langchain.com/docs/integrations/chat/google_generative_ai/
- https://ai.google.dev/pricing?hl=pt-br

Cursos:
- https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/
- https://www.deeplearning.ai/short-courses/langchain-for-llm-application-development/
- https://www.deeplearning.ai/short-courses/langchain-chat-with-your-data/



Criando um chatbot com interface simples:
- https://blog.streamlit.io/how-to-build-an-llm-powered-chatbot-with-streamlit/